### Import Dependencies

In [2]:
!pip install newspaper3k
!pip install --upgrade pip

In [3]:
import os
import pandas as pd
import plotly.graph_objects as go
import random
import newspaper
from newspaper import Article, fulltext, Config, ArticleException
import time
import numpy as np
import requests
import re
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from time import sleep
import threading

In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Define Required Functions

### FY24 ###


This function takes a string argument 'country' and returns a string representing the region the country belongs to.

The regions are defined as follows:

- 'Brazil' for Brazil
- 'Argentina' for Argentina
- 'Mexico' for Mexico
- 'Chile' for Chile
- 'Andean Region' for Colombia, Bolivia, Ecuador, Paraguay, Peru and Uruguay
- 'Central' for Central America countries, Costa Rica, Guatemala, Honduras, Panama, El Salvador, Dominican Republic, Republica Dominicana, Jamaica, Puerto Rico, Trinidad y Tobago and Trinidad and Tobago
- 'Other' for any other country


### COLORS: HEX CODES
- AMAZON : #6B007B
- GOOGLE : #D9B300
- MICROSOFT : #6BA5D9
- SALESFORCE : #AEAEAE

In [6]:
def get_region(country):
    if country == 'Brazil':
        return 'Brazil'
    elif country == 'Argentina':
        #return 'Hispanic South America'
        return 'Spanish South America'
    elif country == 'Mexico':
        return 'Mexico'
    elif country == 'Chile':
        #return 'Hispanic South America'
        return 'Spanish South America'    
    elif country in ['Colombia', 'Bolivia', 'Ecuador', 'Paraguay', 'Peru', 'Uruguay', 'Perú']:
        #return 'Hispanic South America'
        return 'Spanish South America'
    elif country in ['Central', 'Costa Rica', 'Guatemala', 'Honduras', 'Panama', 'El Salvador', 'Dominican Republic', 'Republica Dominicana', 'Jamaica', 'Puerto Rico', 'Trinidad y Tobago', 'Trinidad and Tobago']:
        #return 'Central'
        return 'Central Caribbean'
    else:
        return 'Other'

In [49]:
user = 'katia.bedolla@porternovelli.mx'
API_TOKEN = '7a7668b8111a6e4d5750c12a8c93b56d'

class DiffbotClient(object):
    base_url = 'http://api.diffbot.com/'

    def __init__(self):
        self.session = requests.Session()

    def request(self, url, token, api, fields=None, version=3, **kwargs):
        params = {"url": url, "token": token}
        if fields:
            params['fields'] = fields
        params.update(kwargs)
        response = self.session.get(self.compose_url(api, version), params=params, timeout=10)
        response.raise_for_status()
        return response.json()

    def compose_url(self, api, version_number):
        version = self.format_version_string(version_number)
        return '{}{}/{}'.format(self.base_url, version, api)

    @staticmethod
    def format_version_string(version_number):
        return 'v{}'.format(version_number)

def get_content_diffbot(url):
    diffbot = DiffbotClient()
    token = API_TOKEN
    api = "analyze"
    try:
        response = diffbot.request(url, token, api)
        if 'objects' in response and len(response['objects']) > 0 and 'text' in response['objects'][0]:
            return response['objects'][0]['text']
        else:
            return "No Content"
    except:
        return "Something went wrong with url"


def extract_content_from_urls(urls):
    threads = []
    contents = []
    MAX_CONCURRENCY = 15  # set the maximum number of concurrent threads
    for i in range(0, len(urls), MAX_CONCURRENCY):
        batch = urls[i:i+MAX_CONCURRENCY]
        for url in batch:
            thread = threading.Thread(target=get_content_news, args=(url, contents))
            thread.start()
            threads.append(thread)
        
        for thread in threads:
            thread.join()
        
    return contents

def get_content_news(url, contents):
    user_agent_list = [
   #Chrome
     'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:96.0) Gecko/20100101 Firefox/96.0',

    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]
    user_agent_list = random.choice(user_agent_list)
    user_agent = random.choice(user_agent_list)
    config = Config()
    config.browser_user_agent = user_agent
    try:
        a = Article(url, config=config)
        a.download()
        a.parse()
        paragraphs = a.text
        paragraphs = re.sub(r'<a href=.+?(?=)>|<br/>|\\xa0|\n|</a>|\xa0|<strong>|</strong>|<br/>•|<i(.*?)</i>|<img(.*?)>','',str(paragraphs))

        if isinstance(paragraphs, str) and len(paragraphs) > 0:
            print("URL Content from {} is correct".format(url))
            return paragraphs
        else:
            try:
                ext_diff = get_content_diffbot(url)
                print("URL Content from {} is correct from diffbot")
                return ext_diff
            except Exception as exce:
                print("URL Content from {} is OtherError")
    except ArticleException as e:
        if "Invalid leading whitespace, reserved character(s), or returncharacter(s) in header value: ' '" in str(e):
            print("Error: Invalid header value in article download.")
        elif "HTTPSConnectionPool(host='tecnoblog.net', port=443): Read timed out." in str(e):
            print("Error: Connection timed out while downloading article.")
        elif "403 Client Error: Forbidden" in str(e):
            print("Error: Forbidden to access the article.")
        elif "429 Client Error: Too Many Requests" in str(e):
            print("Error: Too Many Requests, waiting for 5 seconds before trying again.")
            sleep(5)
        elif "Article `download()` failed with HTTPSConnectionPool(host='geekzilla.tech', port=443): Read timed out. (read timeout=7) on URL https://geekzilla.tech/llega-red-hat-ansible-automation-platform-a-google-cloud/" in str(e):
            try:
                ext_diff = get_content_diffbot(url)
                print("URL Content from {} is correct from diffbot".format(url))
                return ext_diff
            except Exception as exce:
                print("URL Content from {} is OtherError")
        else:
            pass

# Define a function to process a single URL
def process_url(url):
    return get_content_news(url, [])

# This function is used to process URLs directly and extract the summary.
def process_url_summary(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        summary = article.summary
        summary = summary.replace("\n", " ")
        print(f'Summary for URL: {url} - Successful')
        return summary
    except:
        print(f'Summary for URL: {url} - Failed')
        return None

In [75]:
def latam_categories(path):
    companies_dfs = []
    for file in os.listdir(path):
        if file.endswith(".csv"):
            platform = file.split("_")[0].capitalize()
            df = pd.read_csv(os.path.join(path, file), sep= None, engine= 'python')
            
            # Add the Company and Category columns
            df['Company'] = file.split('_', 1)[0]
            df['Category'] = file.split('_', 1)[1].split('_', 1)[0]
            
            companies_dfs.append(df)

    return pd.concat(companies_dfs, ignore_index=True)


## AI - LATAM

In [76]:
ai = latam_categories("categories_latam/ai")
ai['Subsidiary']= ai['Country'].apply(get_region)
ai['Category']= 'AI'
ai['Region']= 'LATAM'
print(ai.shape)
ai.head(3)

(2574, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,c17a11aa3fc98c1024cae20f080c6ee8fbd10464d844af...,2023-08-17 13:29:00,IA es un “maremoto” tan grande como internet: ...,https://www.elespectador.com/tecnologia/ia-es-...,elespectador.com,unknown journalist,Colombia,es,329107,3,2,Google,AI,Spanish South America,LATAM
1,7bd7c72adaebfe21a2f49154e2bca8a5b8e04610cf3e28...,2023-08-18 09:00:00,IA Generativa: Google lanza una nueva herramie...,https://www.iproup.com/innovacion/42300-ia-gen...,iproup.com,unknown journalist,Argentina,es,7552,11,2,Google,AI,Spanish South America,LATAM
2,2f09d981e4dfd063a3bd152502768bfae605d08a5c6deb...,2023-08-17 18:35:00,Número de campanhas políticas manipuladas usan...,https://olhardigital.com.br/2023/08/17/pro/num...,olhardigital.com.br,Vitoria Lopes Gomez,Brazil,pt,209607,2,2,Google,AI,Brazil,LATAM


## Brand Innovation - LATAM

In [77]:
brand_innovation = latam_categories("categories_latam/brand_innovation")
brand_innovation['Subsidiary']= brand_innovation['Country'].apply(get_region)
brand_innovation['Category']= 'Brand Innovation'
brand_innovation['Region']= 'LATAM'
print(brand_innovation.shape)
brand_innovation.head()

(1075, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,df0cdb500cf33aa934bb67171cbedfb53e6263598da792...,2023-07-13 14:01:12,Como usar o Google Bard no Brasil? Confira o p...,https://www.tecmundo.com.br/software/266315-co...,tecmundo.com.br,Carlos Palmeira,Brazil,pt,544623,18,2,Google,Brand Innovation,Brazil,LATAM
1,404fa1fa6b6239f27e9ffbac1a33be84341a65b50a149b...,2023-07-18 03:09:00,Esta firma de Wall Street explicó por qué las ...,https://www.forbesargentina.com/money/esta-fir...,forbesargentina.com,Journalist Unknown,Argentina,es,14777,4,2,Google,Brand Innovation,Spanish South America,LATAM
2,fb8683ae06887b63efd7245701ab69fce55dd12af77e50...,2023-07-17 07:20:00,Samsung prueba a introducir ChatGPT en el nave...,https://www.lanacion.com.ar/agencias/samsung-p...,lanacion.com.ar,unknown journalist,Argentina,es,998020,0,1,Google,Brand Innovation,Spanish South America,LATAM
3,37bca161f8f04063d120b48cc6d301efb60fe85a16b295...,2023-07-26 07:10:00,La nueva experiencia de búsqueda de Google bas...,https://www.lanacion.com.ar/agencias/la-nueva-...,lanacion.com.ar,unknown journalist,Argentina,es,1095407,0,2,Google,Brand Innovation,Spanish South America,LATAM
4,295e49c072c7eaf1ac0a5016cacea5c6327d7a050ba046...,2023-07-21 14:52:00,"Na corrida por chips de IA, Google descobre se...",https://olhardigital.com.br/2023/07/21/pro/na-...,olhardigital.com.br,Pedro Borges Spadoni,Brazil,pt,199756,58,2,Google,Brand Innovation,Brazil,LATAM


## Business Applications - LATAM

In [78]:
business_applications = latam_categories("categories_latam/business_applications")
business_applications['Subsidiary']= business_applications['Country'].apply(get_region)
business_applications['Category']= 'Business Applications'
business_applications['Region']= 'LATAM'
print(business_applications.shape)
business_applications.head()

(62, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,807201a4bb672573156b743f0457dad468ea8e1ff23d05...,2023-07-26 17:40:00,"De la mano de la Inteligencia Artificial, tecn...",https://ct.moreover.com/?a=51395965739&p=8vr&v...,iprofesional.com,Adrián Moreno,Argentina,es,155848,2,2,Microsoft,Business Applications,Spanish South America,LATAM
1,b91ece142bf64d6baf4df41d5b2622c1aedcb76789dd86...,2023-09-29 09:42:00,"Caixa, Elo e Microsoft fazem primeira compra e...",https://istoedinheiro.com.br/caixa-elo-e-micro...,istoedinheiro.com.br,Estadão Conteúdo,Brazil,pt,122995,0,1,Microsoft,Business Applications,Brazil,LATAM
2,12db19c9e3075ab33acf2d97dcaf0879c0170d8a20804c...,2023-07-03 11:30:00,5 tendências em alta de colaboração no local d...,https://itforum.com.br/computerworld/5-tendenc...,itforum.com.br,Matthew Finnegan,Brazil,pt,19579,16,2,Microsoft,Business Applications,Brazil,LATAM
3,2d19767e488b65ce71afa2d0a385d2bec75e65de995226...,2023-09-10 10:03:00,Zoom presentó nuevas herramientas de IA Genera...,https://www.forbesargentina.com/innovacion/zoo...,forbesargentina.com,Melody Brue,Argentina,es,18077,5,2,Microsoft,Business Applications,Spanish South America,LATAM
4,c5e03fe62b0b6037e80d8bbb12648b73055840a180d3cc...,2023-09-29 10:57:00,"Caixa, Elo e Microsoft fazem 1ª compra de títu...",https://exame.com/future-of-money/caixa-elo-mi...,exame.com,João Pedro Malar,Brazil,pt,277677,33,1,Microsoft,Business Applications,Brazil,LATAM


## Cloud Platform - LATAM

In [79]:
cloud_platform = latam_categories("categories_latam/cloud_platform")
cloud_platform['Subsidiary']= cloud_platform['Country'].apply(get_region)
cloud_platform['Category']= 'Cloud Platform'
cloud_platform['Region']= 'LATAM'
print(cloud_platform.shape)
cloud_platform.head()

(697, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,686e32410de6e17e4eabecc3dfdf4065133c0c96211373...,2023-07-25 17:50:00,alphabet reporta ganancias superiores a las pr...,https://www.lanacion.com.ar/agencias/alphabet-...,lanacion.com.ar,unknown journalist,Argentina,es,1095407,0,2,Google,Cloud Platform,Spanish South America,LATAM
1,5315d0bfa702b7d87423f1b168afd1366660e3f859349c...,2023-07-26 16:15:37,Google sube 6% en Bolsa por publicidad digital...,https://expansion.mx/mercados/2023/07/26/accio...,expansion.mx,Staff Writer,Mexico,es,163254,3,2,Google,Cloud Platform,Mexico,LATAM
2,e36c6851f319f8ee443a1b183403ca4b7f9cffb8c45d66...,2023-08-29 04:59:00,Inteligencia artificial podrá cubrirte en tus ...,https://www.milenio.com/negocios/inteligencia-...,milenio.com,unknown journalist,Mexico,es,615997,2,2,Google,Cloud Platform,Mexico,LATAM
3,f07f7b6957736cfb7a231320c79b6d10418a7dcd6fc76f...,2023-08-31 00:07:00,Google Cloud establecerá operaciones en El Sal...,https://dplnews.com/google-cloud-establecera-o...,dplnews.com,Dinorah Navarro,Mexico,es,44797,2,2,Google,Cloud Platform,Mexico,LATAM
4,1b0e55bcbaca121fa411fda76dedae713924112d5bff42...,2023-09-04 12:24:00,Continental vai integrar serviços do Google Cl...,http://forbes.com.br/forbes-tech/2023/09/conti...,forbes.com.br,Fernanda Almeida,Brazil,pt,55801,2,2,Google,Cloud Platform,Brazil,LATAM


## Developers - LATAM

In [80]:
developers = latam_categories("categories_latam/developers")
developers['Subsidiary']= developers['Country'].apply(get_region)
developers['Region']= 'LATAM'
print(developers.shape)
developers.head()

(157, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,c8c49005158027f58df3198819e503989430d7ee16021f...,2023-07-27 07:23:00,AWS lanza siete nuevos cursos para aprender IA...,https://portalinnova.cl/aws-lanza-siete-nuevos...,portalinnova.cl,unknown journalist,Chile,es,4672,0,2,Amazon,Developers,Spanish South America,LATAM
1,59e57a1ad35c6f5f24716ab1a289e853185e83dff80c54...,2023-08-01 20:16:00,Acumulando polvo y datos: cómo las aspiradoras...,https://portalinnova.cl/acumulando-polvo-y-dat...,portalinnova.cl,unknown journalist,Chile,es,4672,2,2,Amazon,Developers,Spanish South America,LATAM
2,aa4e6c002b7b27a80321f9aff89763c5b59a4a4a35ca48...,2023-09-28 14:00:00,AWS anuncia novas ofertas para acelerar a adoç...,https://inforchannel.com.br/2023/09/28/aws-anu...,inforchannel.com.br,unknown journalist,Brazil,pt,8194,0,2,Amazon,Developers,Brazil,LATAM
3,162bb80c94fdcec99946240b58f83d73635ef3ca833af2...,2023-07-30 05:13:00,Cómo aprender sobre IA generativa con estos si...,https://www.forbesargentina.com/innovacion/com...,forbesargentina.com,Nicolás Della Vecchia,Argentina,es,18077,9,2,Amazon,Developers,Spanish South America,LATAM
4,6ff33081a70ee1e3287ce37ce75486761158a64d864390...,2023-09-29 08:25:00,AWS anuncia nuevos modelos fundacionales y cap...,https://www.eltiempo.com/tecnosfera/novedades-...,eltiempo.com,unknown journalist,Colombia,es,726205,8,2,Amazon,Developers,Spanish South America,LATAM


## Productivity - LATAM

In [81]:
productivity = latam_categories("categories_latam/productivity")
productivity['Subsidiary']= productivity['Country'].apply(get_region)
productivity['Region']= 'LATAM'
print(productivity.shape)
productivity.head()

(832, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,2700d0349a578c6fac40a53295277d0491224e581afc89...,2023-07-03 07:00:00,Microsoft enfrenta otra investigación antimono...,https://expansion.mx/tecnologia/2023/07/03/mic...,expansion.mx,unknown journalist,Mexico,es,163254,3,2,Salesforce,Productivity,Mexico,LATAM
1,4ba5602bb70343afe867963803eed3089d1808454627f0...,2023-07-26 01:01:00,"Lidiane Jones: “Se você sempre estiver certo, ...",https://epocanegocios.globo.com/personalidades...,epocanegocios.globo.com,Andressa Rovani,Brazil,pt,132427,5,2,Salesforce,Productivity,Brazil,LATAM
2,bb033133770d5abc888ed5e21a5ae08a4b22d7599780c0...,2023-08-09 09:50:00,Repaginada no Slack deixa interface mais 'clea...,https://olhardigital.com.br/2023/08/09/reviews...,olhardigital.com.br,Borges Spadoni,Brazil,pt,209607,1,2,Salesforce,Productivity,Brazil,LATAM
3,80fed41c388cc62635681e38cf79d814bbe3d5973f2167...,2023-09-21 12:18:00,"IA tornará o trabalho mais humano, diz CEO do ...",https://olhardigital.com.br/2023/09/21/pro/ia-...,olhardigital.com.br,Alessandro Di Lorenzo,Brazil,pt,209607,20,2,Salesforce,Productivity,Brazil,LATAM
4,52d23dc11c0d1dd75c4b73a7865ae4bde6c449efbb4b9f...,2023-08-02 13:00:00,Salesforce lança Slack Sales Elevate para ajud...,https://inforchannel.com.br/2023/08/02/salesfo...,inforchannel.com.br,unknown journalist,Brazil,pt,8194,0,2,Salesforce,Productivity,Brazil,LATAM


## Security - LATAM

In [82]:
security = latam_categories("categories_latam/security")
security['Subsidiary']= security['Country'].apply(get_region)
security['Region']= 'LATAM'
print(security.shape)
security.head()

(895, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,d1412badf223691ee0317d2f3f69d1f99096482de44bad...,2023-08-08 14:18:00,Android 14 terá melhorias de segurança “inédit...,https://canaltech.com.br/android/android-14-te...,canaltech.com.br,Felipe Demartini,Brazil,pt,338531,7,2,Google,Security,Brazil,LATAM
1,aaffdfbc0dc5d5953771151f89a648ee2f5f38dfca32a5...,2023-08-16 02:00:00,Google Drive: Cómo proteger y encriptar los da...,https://www.estrategiaynegocios.net/tecnologia...,estrategiaynegocios.net,unknown journalist,El Salvador,es,26019,3,2,Google,Security,Central Caribbean,LATAM
2,74b032edc0a419884b4ebc65a047ff1b1b4c71a0e40241...,2023-08-30 09:20:00,Con este trucazo podrá saber si su correo de G...,https://www.semana.com/tecnologia/articulo/con...,semana.com,unknown journalist,Colombia,es,291733,3,2,Google,Security,Spanish South America,LATAM
3,2599c6fa3bb0b4c292fdd50d3826782f6c7fa244912723...,2023-09-08 14:41:43,"A qué se dedican los hackers rojos, los guardi...",https://www.infobae.com/tecno/2023/09/08/a-que...,infobae.com,unknown journalist,Argentina,es,1529212,37,2,Google,Security,Spanish South America,LATAM
4,47dc98df86b3abbd8a70ecb619d0cc8a5a8366b1536bba...,2023-09-13 19:09:58,"Falha de segurança afeta Chrome, Firefox, Edge...",https://tecnoblog.net/noticias/2023/09/13/falh...,tecnoblog.net,Giovanni Santa Rosa,Brazil,pt,175722,28,2,Google,Security,Brazil,LATAM


## Sustainability - LATAM

In [83]:
sustainability = latam_categories("categories_latam/sustainability")
sustainability['Subsidiary']= sustainability['Country'].apply(get_region)
sustainability['Region']= 'LATAM'
print(sustainability.shape)
sustainability.head()

(105, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,31dbb5315a47f03ea2829385974cdeab5549ed5cd7d5f6...,2023-09-05 13:00:00,Microsoft registra patente de mochila intelige...,https://epocanegocios.globo.com/tecnologia/not...,epocanegocios.globo.com,Soraia Alves,Brazil,pt,132427,1,2,Microsoft,Sustainability,Brazil,LATAM
1,365a4b427fc0ab2b0f74397ebe6664d9cf7c3c384c2355...,2023-08-31 12:12:07,Microsoft busca ideas innovadoras para protege...,https://www.elnuevodia.com/brandstudio-publish...,elnuevodia.com,unknown journalist,Puerto Rico,es,264700,45,2,Microsoft,Sustainability,Central Caribbean,LATAM
2,e219835df632d818a7c194d8a0ebc2a70afb7c17fef9c3...,2023-07-10 07:00:00,Microsoft y sus ambiciosas metas en materia de...,https://impactotic.co/microsoft-y-sus-metas-de...,impactotic.co,Daniela Jaramillo Castillo,Colombia,es,8951,17,2,Microsoft,Sustainability,Spanish South America,LATAM
3,2ea9ab60d7255e7ac874817a972326d6a595c1e8fe8477...,2023-09-04 09:43:59,A inteligência artificial vai piorar as mudanç...,https://www.estadao.com.br/sustentabilidade/a-...,estadao.com.br,unknown journalist,Brazil,pt,597225,19,2,Microsoft,Sustainability,Brazil,LATAM
4,7af3d70bc23291faa0ebfcbc9de28a7e3c8a6751ca1731...,2023-09-28 21:37:00,Microsoft busca concretos mais sustentáveis pa...,https://olhardigital.com.br/2023/09/28/pro/mic...,olhardigital.com.br,Rodrigo Mozelli,Brazil,pt,209607,1,2,Microsoft,Sustainability,Brazil,LATAM


## Trust Overview - LATAM

In [84]:
trust_overview = latam_categories("categories_latam/trust_overview")
trust_overview['Subsidiary']= trust_overview['Country'].apply(get_region)
trust_overview['Category']= 'Trust Overview'
trust_overview['Region']= 'LATAM'
print(trust_overview.shape)
trust_overview.head()

(3883, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,9b783b78c5773153e76592fbdcd3fe34bbe3e2610da10c...,2023-09-11 09:30:00,Focos inteligentes compatibles con Alexa: con ...,https://www.xataka.com.mx/seleccion/focos-inte...,xataka.com.mx,Alejandro Aranda,Mexico,es,58968,0.0,2,Amazon,Trust Overview,Mexico,LATAM
1,53ee0884a61d9254f495c9830de47a7a1abaecf7068689...,2023-09-15 18:00:00,¡Oficialmente llega a México el programa educa...,https://geekzilla.tech/oficialmente-llega-a-me...,geekzilla.tech,unknown journalist,Mexico,es,2731,0.0,2,Amazon,Trust Overview,Mexico,LATAM
2,71d72de811e5d0d1df925547d77dd05f650ff7f3ecfd66...,2023-09-26 11:48:00,"Crítica das Big Tech, presidente da Comissão d...",https://epocanegocios.globo.com/reuters/notici...,epocanegocios.globo.com,Unknow Jornalist,Brazil,pt,132427,2.0,2,Amazon,Trust Overview,Brazil,LATAM
3,289fc1b7e4ea163d569ff0f533145cf7b2ebea49d55af4...,2023-09-25 05:39:00,Amazon investe até US$ 4 bilhões em empresa de IA,https://exame.com/inteligencia-artificial/amaz...,exame.com,unknown journalist,Brazil,pt,277677,2.0,2,Amazon,Trust Overview,Brazil,LATAM
4,7a33e5e4067e5770be1abf52b17ff3b22b12e82ffef3f1...,2023-09-26 14:21:00,"Crítica das Big Tech, presidente da Comissão d...",https://www.uol.com.br/tilt/noticias/reuters/2...,uol.com.br,David Shepardson,Brazil,pt,4794257,0.0,2,Amazon,Trust Overview,Brazil,LATAM


## Concatenate All LATAM DataFrames

In [86]:
latam_q1 = pd.concat([ai, brand_innovation, business_applications, cloud_platform,
                     developers, productivity, security, sustainability, trust_overview])
print(latam_q1.shape)
latam_q1.head(3)

(10280, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,c17a11aa3fc98c1024cae20f080c6ee8fbd10464d844af...,2023-08-17 13:29:00,IA es un “maremoto” tan grande como internet: ...,https://www.elespectador.com/tecnologia/ia-es-...,elespectador.com,unknown journalist,Colombia,es,329107,3.0,2,Google,AI,Spanish South America,LATAM
1,7bd7c72adaebfe21a2f49154e2bca8a5b8e04610cf3e28...,2023-08-18 09:00:00,IA Generativa: Google lanza una nueva herramie...,https://www.iproup.com/innovacion/42300-ia-gen...,iproup.com,unknown journalist,Argentina,es,7552,11.0,2,Google,AI,Spanish South America,LATAM
2,2f09d981e4dfd063a3bd152502768bfae605d08a5c6deb...,2023-08-17 18:35:00,Número de campanhas políticas manipuladas usan...,https://olhardigital.com.br/2023/08/17/pro/num...,olhardigital.com.br,Vitoria Lopes Gomez,Brazil,pt,209607,2.0,2,Google,AI,Brazil,LATAM


In [87]:
print(latam_q1['Category'].value_counts())
print('\n')
print(latam_q1['Company'].value_counts())

Trust Overview           3883
AI                       2574
Brand Innovation         1075
Security                  895
Productivity              832
Cloud Platform            697
Developers                157
Sustainability            105
Business Applications      62
Name: Category, dtype: int64


Google        5034
Microsoft     4066
Amazon        1019
Salesforce     148
Zoom            13
Name: Company, dtype: int64


## AI - Countries

In [88]:
ai_countries = latam_categories("categories_countries/ai")
ai_countries['Subsidiary']= ai_countries['Country'].apply(get_region)
ai_countries['Category']= 'AI'
ai_countries['Region']= 'COUNTRIES'
print(ai_countries.shape)
ai_countries.head(3)

(2878, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,c8c49005158027f58df3198819e503989430d7ee16021f...,2023-07-27 07:23:00,AWS lanza siete nuevos cursos para aprender IA...,https://portalinnova.cl/aws-lanza-siete-nuevos...,portalinnova.cl,unknown journalist,Chile,es,4672,0,2,Amazon,AI,Spanish South America,COUNTRIES
1,bac1b1365d3347b0c72135e102e11a7ead5b378fc3f26a...,2023-09-20 19:15:53,"Alexa ""casi humana"": Amazon presenta nueva ver...",https://www.df.cl/internacional/multinacionale...,df.cl,unknown journalist,Chile,es,45232,16,2,Amazon,AI,Spanish South America,COUNTRIES
2,9fadbc44ddb0af58e501328f0255e25a04a89ab5d778dc...,2023-09-06 08:16:00,Grupo BMW selecciona a AWS para impulsar la pl...,https://portalinnova.cl/grupo-bmw-selecciona-a...,portalinnova.cl,Journalist Unknown,Chile,es,4672,0,2,Amazon,AI,Spanish South America,COUNTRIES


## Brand Innovation - Countries

In [89]:
brand_innovation_countries = latam_categories("categories_countries/brand_innovation")
brand_innovation_countries['Subsidiary']= brand_innovation_countries['Country'].apply(get_region)
brand_innovation_countries['Category']= 'Brand Innovation'
brand_innovation_countries['Region']= 'COUNTRIES'
print(brand_innovation_countries.shape)
brand_innovation_countries.head(3)

(1401, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,1375b86f1ac844308c2cf0fc62eaa5674a7e5f5f17fdc1...,2023-08-30 10:51:00,7 aplicaciones para hacer traducción en tiempo...,https://www.lanacion.com.ar/tecnologia/7-aplic...,lanacion.com.ar,Desirée Jaimovich,Argentina,es,1095407,55,2,Google,Brand Innovation,Spanish South America,COUNTRIES
1,a1936cd57045b9a503305cf6b3bcfe319a1312249f7268...,2023-07-09 23:09:47,Las películas más populares en Google Colombia,https://www.infobae.com/noticias/2023/07/10/la...,infobae.com,Newsroom Infobae,Argentina,es,1442272,0,2,Google,Brand Innovation,Spanish South America,COUNTRIES
2,61e8a16e19535035d2e2128b81d4940d1306a3498c4143...,2023-08-15 15:32:59,Google Slides: cómo usar gratis esta herramien...,https://www.infobae.com/tecno/2023/08/15/googl...,infobae.com,Jimmy Pepinosa,Argentina,es,1529212,7,2,Google,Brand Innovation,Spanish South America,COUNTRIES


## Business Applications - Countries

In [90]:
business_applications_countries = latam_categories("categories_countries/business_applications")
business_applications_countries['Subsidiary']= business_applications_countries['Country'].apply(get_region)
business_applications_countries['Category']= 'Business Applications'
business_applications_countries['Region']= 'COUNTRIES'
print(business_applications_countries.shape)
business_applications_countries.head(3)

(68, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,4a4a53ed77176439ce8bee1af7eff485b61c35a0dccd95...,2023-08-28 17:32:00,"Salesforce presenta Einstein GPT, la IA genera...",https://www.enter.co/especiales/dev/ai/salesfo...,enter.co,Digna Irene Urrea,Colombia,es,114077,16,2,Salesforce,Business Applications,Spanish South America,COUNTRIES
1,d8c2d212c4c846d1264c123a46a6f04f517cddf761a664...,2023-09-28 03:55:15,Salesforce realizó su evento Dreamforce que ge...,https://www.larepublica.co/empresas/salesforce...,larepublica.co,Erika Viviana Ruiz,Colombia,es,78563,2,2,Salesforce,Business Applications,Spanish South America,COUNTRIES
2,33f9ca67523921fa24330d97b460dc03373952bf09ff2a...,2023-09-13 18:06:00,Colsubsidio abrió ofertas de trabajo en Bogotá...,https://www.semana.com/finanzas/articulo/colsu...,semana.com,unknown journalist,Colombia,es,291733,16,2,Salesforce,Business Applications,Spanish South America,COUNTRIES


## Cloud Platform - Countries

In [91]:
cloud_platform_countries = latam_categories("categories_countries/cloud_platform")
cloud_platform_countries['Subsidiary']= cloud_platform_countries['Country'].apply(get_region)
cloud_platform_countries['Category']= 'Cloud Platform'
cloud_platform_countries['Region']= 'COUNTRIES'
print(cloud_platform_countries.shape)
cloud_platform_countries.head(3)

(758, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,864d714ccc322019b1bc51e024cc7bc0ef7746520fbfe8...,2023-08-20 22:17:23,Se expande,https://www.eleconomista.com.mx/opinion/Se-exp...,eleconomista.com.mx,unknown journalist,Mexico,es,207936,0,2,Salesforce,Cloud Platform,Mexico,COUNTRIES
1,cb71c7b2073a575cceb6a19bfd4c1605231d320e67a348...,2023-09-14 16:43:32,Salesforce prepara Einstein Copilot para liber...,https://cio.com.mx/salesforce-prepara-einstein...,cio.com.mx,By José Luis Becerra Pozas,Mexico,es,9855,7,2,Salesforce,Cloud Platform,Mexico,COUNTRIES
2,f1b6e6e8d50cf2b05311b80bb9aa70b33994a97a7b314b...,2023-09-12 10:35:03,Salesforce anuncia su nuevo asistente con inte...,https://www.infobae.com/espana/agencias/2023/0...,infobae.com,unknown journalist,Argentina,es,1529212,1,2,Salesforce,Cloud Platform,Spanish South America,COUNTRIES


## Developers - Countries

In [92]:
developers_countries = latam_categories("categories_countries/developers")
developers_countries['Subsidiary']= developers_countries['Country'].apply(get_region)
developers_countries['Category']= 'Developers'
developers_countries['Region']= 'COUNTRIES'
print(developers_countries.shape)
developers_countries.head(3)

(182, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,b27ada94116c8d0f10e7fdfe39325821696cdcf030cec4...,2023-07-06 09:00:00,Aquisição da iRobot pela Amazon entra na mira ...,https://epocanegocios.globo.com/empresas/notic...,epocanegocios.globo.com,unknown journalist,Brazil,pt,120523,2,2,Amazon,Developers,Brazil,COUNTRIES
1,a4d574466d7554ddac813f87b2f06e2e0c3eb95fef91ac...,2023-09-25 04:27:00,Amazon vai investir até US$ 4 bilhões em start...,https://canaltech.com.br/mercado/amazon-vai-in...,canaltech.com.br,André Lourenti Magalhães,Brazil,pt,338531,5,2,Amazon,Developers,Brazil,COUNTRIES
2,7c807ef23daf2e890963aabf38ceb06ecda3eb3baa606b...,2023-08-22 10:00:00,Novos recursos do VMware Cloud para modernizar...,https://inforchannel.com.br/2023/08/22/novos-r...,inforchannel.com.br,unknown journalist,Brazil,pt,8194,0,2,Amazon,Developers,Brazil,COUNTRIES


## Productivity - Countries

In [93]:
productivity_countries = latam_categories("categories_countries/productivity")
productivity_countries['Subsidiary']= productivity_countries['Country'].apply(get_region)
productivity_countries['Category']= 'Productivity'
productivity_countries['Region']= 'COUNTRIES'
print(productivity_countries.shape)
productivity_countries.head(3)

(924, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,ba47b59fb590948ef3fc1ca2d13b01fd23fb8b3329f4e7...,2023-08-14 07:51:00,Advogados pedem explicação sobre aplicativo in...,https://epocanegocios.globo.com/brasil/noticia...,epocanegocios.globo.com,unknown journalist,Brazil,pt,132427,2,2,Google,Productivity,Brazil,COUNTRIES
1,da8ecb06c7de591a136eb7ab7fdc55f1989e0c1c38b1a0...,2023-08-29 12:16:00,Nova IA do Google Meet pode participar de reun...,https://olhardigital.com.br/2023/08/29/pro/nov...,olhardigital.com.br,Gabriel Sérvio,Brazil,pt,209607,9,2,Google,Productivity,Brazil,COUNTRIES
2,94b45e5009f7f24953e3ad6a6f1a2231e140f4386c521b...,2023-08-25 15:55:00,Google Keep finalmente permite anotações com t...,https://olhardigital.com.br/2023/08/25/interne...,olhardigital.com.br,André Fogaça,Brazil,pt,209607,20,2,Google,Productivity,Brazil,COUNTRIES


## Security - Countries

In [94]:
security_countries = latam_categories("categories_countries/security")
security_countries['Subsidiary']= security_countries['Country'].apply(get_region)
security_countries['Category']= 'Security'
security_countries['Region']= 'COUNTRIES'
print(security_countries.shape)
security_countries.head(3)

(949, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,e0c207b898f0d2f153ad9d12cad6c562fa26b04a040aae...,2023-08-31 00:34:00,Microsoft separará Teams de Office ante preocu...,https://www.estrategiaynegocios.net/empresasym...,estrategiaynegocios.net,Journalist Unknown,El Salvador,es,26019,22,2,Microsoft,Security,Central Caribbean,COUNTRIES
1,9149ae2d70c019340ca9ccce39a4cf02ee6006ec4d3956...,2023-07-19 15:05:00,Microsoft regalará herramientas de ciberseguridad,https://ct.moreover.com/?a=51339103858&p=8vr&v...,estrategiaynegocios.net,Estrategia y Negocios,El Salvador,es,26019,2,2,Microsoft,Security,Central Caribbean,COUNTRIES
2,818ca27c445bacee3abe2dd7d9673f84040b87d1d93c35...,2023-07-27 13:26:00,UE abre investigación contra Microsoft por prá...,https://ct.moreover.com/?a=51404790643&p=8vr&v...,estrategiaynegocios.net,Estrategia y Negocios,El Salvador,es,26019,6,2,Microsoft,Security,Central Caribbean,COUNTRIES


## Sustainability - Countries

In [95]:
sustainability_countries = latam_categories("categories_countries/sustainability")
sustainability_countries['Subsidiary']= sustainability_countries['Country'].apply(get_region)
sustainability_countries['Category']= 'Sustainability'
sustainability_countries['Region']= 'COUNTRIES'
print(sustainability_countries.shape)
sustainability_countries.head(3)

(126, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,d0067d987867c0e1a839f065aada870cd483d9a3afa884...,2023-08-29 21:03:00,Google lanza nuevas herramientas para ayudar a...,https://lopezdoriga.com/ciencia-tecnologia/goo...,lopezdoriga.com,unknown journalist,Mexico,es,53854,4,2,Google,Sustainability,Mexico,COUNTRIES
1,51dbfc4d9873fb0b9a15ec43212e0997d523eb8f4c1888...,2023-07-30 14:15:00,Inteligencia Artificial al servicio del campo ...,https://dplnews.com/inteligencia-artificial-al...,dplnews.com,Sharon Durán,Mexico,es,44797,23,2,Google,Sustainability,Mexico,COUNTRIES
2,cbbdee5f13ea22b3e70d818dbf00399810ede6ecbd45b5...,2023-07-26 06:30:00,El consumo de agua de Google se dispara y la I...,https://businessinsider.mx/consumo-agua-google...,businessinsider.mx,Hugh Langley,Mexico,es,68346,3,2,Google,Sustainability,Mexico,COUNTRIES


## Trust Overview - Countries

In [96]:
trust_overview_countries = latam_categories("categories_countries/trust_overview")
trust_overview_countries['Subsidiary']= trust_overview_countries['Country'].apply(get_region)
trust_overview_countries['Category']= 'Trust Overview'
trust_overview_countries['Region']= 'COUNTRIES'
print(trust_overview_countries.shape)
trust_overview_countries.head(3)

(4330, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,2f658d231d016f127266cc10ae144789fd4d8d59810a9e...,2023-07-26 12:24:06,Governo determina que Facebook e Google remova...,https://www.uol.com.br/tilt/ultimas-noticias/e...,uol.com.br,Natália Santos,Brazil,pt,4794257,0.0,2,Google,Trust Overview,Brazil,COUNTRIES
1,eeefcfb9ed881c62da57ce754c40ad2dbddf5b66135f3f...,2023-07-26 03:06:00,"Alta de juros do Fed, lucro do Santander afund...",https://exame.com/invest/mercados/alta-de-juro...,exame.com,unknown journalist,Brazil,pt,277677,11.0,2,Google,Trust Overview,Brazil,COUNTRIES
2,50242c025170f7ec6c39c1732526a5503f24f802672321...,2023-08-06 09:06:00,Celular cheio? Veja como excluir mídia indesej...,https://gizmodo.uol.com.br/celular-cheio-veja-...,gizmodo.uol.com.br,Gabriel Andrade,Brazil,pt,197617,0.0,2,Google,Trust Overview,Brazil,COUNTRIES


## Concatenate All Countries DataFrames

In [97]:
countries_q1 = pd.concat([ai_countries, brand_innovation_countries, business_applications_countries, cloud_platform_countries,
                     developers_countries, productivity_countries, security_countries, sustainability_countries, trust_overview_countries])
print(countries_q1.shape)
countries_q1.head(3)

(11616, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,c8c49005158027f58df3198819e503989430d7ee16021f...,2023-07-27 07:23:00,AWS lanza siete nuevos cursos para aprender IA...,https://portalinnova.cl/aws-lanza-siete-nuevos...,portalinnova.cl,unknown journalist,Chile,es,4672,0.0,2,Amazon,AI,Spanish South America,COUNTRIES
1,bac1b1365d3347b0c72135e102e11a7ead5b378fc3f26a...,2023-09-20 19:15:53,"Alexa ""casi humana"": Amazon presenta nueva ver...",https://www.df.cl/internacional/multinacionale...,df.cl,unknown journalist,Chile,es,45232,16.0,2,Amazon,AI,Spanish South America,COUNTRIES
2,9fadbc44ddb0af58e501328f0255e25a04a89ab5d778dc...,2023-09-06 08:16:00,Grupo BMW selecciona a AWS para impulsar la pl...,https://portalinnova.cl/grupo-bmw-selecciona-a...,portalinnova.cl,Journalist Unknown,Chile,es,4672,0.0,2,Amazon,AI,Spanish South America,COUNTRIES


In [98]:
print(countries_q1['Category'].value_counts())
print('\n')
print(countries_q1['Company'].value_counts())

Trust Overview           4330
AI                       2878
Brand Innovation         1401
Security                  949
Productivity              924
Cloud Platform            758
Developers                182
Sustainability            126
Business Applications      68
Name: Category, dtype: int64


Google        5610
Microsoft     4546
Amazon        1276
Salesforce     166
Zoom            18
Name: Company, dtype: int64


## Date Details - LATAM

In [129]:
latam_q1['Published date'] = pd.to_datetime(latam_q1['Published date'])

latam_q1['Day'] = latam_q1['Published date'].dt.strftime('%d')
latam_q1['Month'] = latam_q1['Published date'].dt.strftime('%B')
latam_q1['Year'] = latam_q1['Published date'].dt.strftime('%Y')
print(latam_q1.shape)
latam_q1.head()

(10280, 18)


,"﻿""Article ID""",Published date,Headline,Url,Outlet,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region,Day,Month,Year
0,c17a11aa3fc98c1024cae20f080c6ee8fbd10464d844af...,2023-08-17 13:29:00,IA es un “maremoto” tan grande como internet: ...,https://www.elespectador.com/tecnologia/ia-es-...,elespectador.com,unknown journalist,Colombia,es,329107,3.0,2,Google,AI,Spanish South America,LATAM,17,August,2023
1,7bd7c72adaebfe21a2f49154e2bca8a5b8e04610cf3e28...,2023-08-18 09:00:00,IA Generativa: Google lanza una nueva herramie...,https://www.iproup.com/innovacion/42300-ia-gen...,iproup.com,unknown journalist,Argentina,es,7552,11.0,2,Google,AI,Spanish South America,LATAM,18,August,2023
2,2f09d981e4dfd063a3bd152502768bfae605d08a5c6deb...,2023-08-17 18:35:00,Número de campanhas políticas manipuladas usan...,https://olhardigital.com.br/2023/08/17/pro/num...,olhardigital.com.br,Vitoria Lopes Gomez,Brazil,pt,209607,2.0,2,Google,AI,Brazil,LATAM,17,August,2023
3,b56b1ddf9e15b478f66cbb0b395f3eb2888cedfcf95b44...,2023-08-11 09:25:00,IA generativa: Magalu e Dasa apresentam projet...,https://itforum.com.br/noticias/ia-generativa-...,itforum.com.br,Marcelo Gimenes Vieira,Brazil,pt,21534,11.0,2,Google,AI,Brazil,LATAM,11,August,2023
4,ed2b4ac6329dd3d77ea6400107cd1f4f0779bf5a568692...,2023-08-22 17:09:43,Google se la juega: Bard reveló cuál es la mej...,https://www.iproup.com/economia-digital/42067-...,iproup.com,unknown journalist,Argentina,es,7552,32.0,2,Google,AI,Spanish South America,LATAM,22,August,2023


## Merge With Media List DF

In [130]:
media_list_fy24= pd.read_excel(r"FY24_media_list_demo.xlsx", sheet_name= "FY24 Media List Americas")
media_list_fy24= media_list_fy24[['Outlet', 'Media Type', 'Region', 'Women', 'Men', '18-24', '25-34', '35-44', '45+']]
print(media_list_fy24.shape)
media_list_fy24.head()

(146, 9)


,Outlet,Media Type,Region,Women,Men,18-24,25-34,35-44,45+
0,adrenaline.com.br,Consumer,Americas,0.187832,0.812168,0.353864,0.354373,0.149176,0.142587
1,advisor.ca,Business,Americas,NaN,NaN,NaN,NaN,NaN,NaN
2,ambito.com,Business,Americas,0.372917,0.627083,0.154652,0.287253,0.207571,0.350524
3,andina.pe,Business,Americas,0.478957,0.521043,0.302806,0.302415,0.172419,0.222360
4,automationmag.com,Tech,Americas,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
# Change the column name "Outlet Name" of latam_q1 dataframe to "Outlet"
latam_q1.rename(columns={'Outlet Name':'Outlet'}, inplace=True)
print(latam_q1.shape)
latam_q1.head()

(10280, 18)


,"﻿""Article ID""",Published date,Headline,Url,Outlet,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region,Day,Month,Year
0,c17a11aa3fc98c1024cae20f080c6ee8fbd10464d844af...,2023-08-17 13:29:00,IA es un “maremoto” tan grande como internet: ...,https://www.elespectador.com/tecnologia/ia-es-...,elespectador.com,unknown journalist,Colombia,es,329107,3.0,2,Google,AI,Spanish South America,LATAM,17,August,2023
1,7bd7c72adaebfe21a2f49154e2bca8a5b8e04610cf3e28...,2023-08-18 09:00:00,IA Generativa: Google lanza una nueva herramie...,https://www.iproup.com/innovacion/42300-ia-gen...,iproup.com,unknown journalist,Argentina,es,7552,11.0,2,Google,AI,Spanish South America,LATAM,18,August,2023
2,2f09d981e4dfd063a3bd152502768bfae605d08a5c6deb...,2023-08-17 18:35:00,Número de campanhas políticas manipuladas usan...,https://olhardigital.com.br/2023/08/17/pro/num...,olhardigital.com.br,Vitoria Lopes Gomez,Brazil,pt,209607,2.0,2,Google,AI,Brazil,LATAM,17,August,2023
3,b56b1ddf9e15b478f66cbb0b395f3eb2888cedfcf95b44...,2023-08-11 09:25:00,IA generativa: Magalu e Dasa apresentam projet...,https://itforum.com.br/noticias/ia-generativa-...,itforum.com.br,Marcelo Gimenes Vieira,Brazil,pt,21534,11.0,2,Google,AI,Brazil,LATAM,11,August,2023
4,ed2b4ac6329dd3d77ea6400107cd1f4f0779bf5a568692...,2023-08-22 17:09:43,Google se la juega: Bard reveló cuál es la mej...,https://www.iproup.com/economia-digital/42067-...,iproup.com,unknown journalist,Argentina,es,7552,32.0,2,Google,AI,Spanish South America,LATAM,22,August,2023


In [132]:
latam_q1_processed = latam_q1.merge(media_list_fy24, on='Outlet', how='left')
print(latam_q1_processed.shape)
latam_q1_processed.head()

(10280, 26)


,"﻿""Article ID""",Published date,Headline,Url,Outlet,Author Name,Country,Language,Impressions,Shares,...,Month,Year,Media Type,Region_y,Women,Men,18-24,25-34,35-44,45+
0,c17a11aa3fc98c1024cae20f080c6ee8fbd10464d844af...,2023-08-17 13:29:00,IA es un “maremoto” tan grande como internet: ...,https://www.elespectador.com/tecnologia/ia-es-...,elespectador.com,unknown journalist,Colombia,es,329107,3.0,...,August,2023,General_News,Americas,0.458474,0.541526,0.255116,0.350201,0.170343,0.224340
1,7bd7c72adaebfe21a2f49154e2bca8a5b8e04610cf3e28...,2023-08-18 09:00:00,IA Generativa: Google lanza una nueva herramie...,https://www.iproup.com/innovacion/42300-ia-gen...,iproup.com,unknown journalist,Argentina,es,7552,11.0,...,August,2023,Tech,Americas,0.322019,0.677981,0.161540,0.328019,0.223239,0.287202
2,2f09d981e4dfd063a3bd152502768bfae605d08a5c6deb...,2023-08-17 18:35:00,Número de campanhas políticas manipuladas usan...,https://olhardigital.com.br/2023/08/17/pro/num...,olhardigital.com.br,Vitoria Lopes Gomez,Brazil,pt,209607,2.0,...,August,2023,Tech,Americas,0.361911,0.638089,0.255087,0.348724,0.178670,0.217519
3,b56b1ddf9e15b478f66cbb0b395f3eb2888cedfcf95b44...,2023-08-11 09:25:00,IA generativa: Magalu e Dasa apresentam projet...,https://itforum.com.br/noticias/ia-generativa-...,itforum.com.br,Marcelo Gimenes Vieira,Brazil,pt,21534,11.0,...,August,2023,Tech,Americas,0.360594,0.639406,0.244546,0.391039,0.200255,0.164160
4,ed2b4ac6329dd3d77ea6400107cd1f4f0779bf5a568692...,2023-08-22 17:09:43,Google se la juega: Bard reveló cuál es la mej...,https://www.iproup.com/economia-digital/42067-...,iproup.com,unknown journalist,Argentina,es,7552,32.0,...,August,2023,Tech,Americas,0.322019,0.677981,0.161540,0.328019,0.223239,0.287202


## Date Details - Countries

In [103]:
countries_q1['Published date'] = pd.to_datetime(countries_q1['Published date'])

countries_q1['Day'] = countries_q1['Published date'].dt.strftime('%d')
countries_q1['Month'] = countries_q1['Published date'].dt.strftime('%B')
countries_q1['Year'] = countries_q1['Published date'].dt.strftime('%Y')
print(countries_q1.shape)
countries_q1.head()

(11616, 18)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region,Day,Month,Year
0,c8c49005158027f58df3198819e503989430d7ee16021f...,2023-07-27 07:23:00,AWS lanza siete nuevos cursos para aprender IA...,https://portalinnova.cl/aws-lanza-siete-nuevos...,portalinnova.cl,unknown journalist,Chile,es,4672,0.0,2,Amazon,AI,Spanish South America,COUNTRIES,27,July,2023
1,bac1b1365d3347b0c72135e102e11a7ead5b378fc3f26a...,2023-09-20 19:15:53,"Alexa ""casi humana"": Amazon presenta nueva ver...",https://www.df.cl/internacional/multinacionale...,df.cl,unknown journalist,Chile,es,45232,16.0,2,Amazon,AI,Spanish South America,COUNTRIES,20,September,2023
2,9fadbc44ddb0af58e501328f0255e25a04a89ab5d778dc...,2023-09-06 08:16:00,Grupo BMW selecciona a AWS para impulsar la pl...,https://portalinnova.cl/grupo-bmw-selecciona-a...,portalinnova.cl,Journalist Unknown,Chile,es,4672,0.0,2,Amazon,AI,Spanish South America,COUNTRIES,06,September,2023
3,6d4a013c3b84331acf1669b171ca9cf5ab7c06f384d7d9...,2023-09-26 17:53:09,Amazon y Anthropic colaboran en el avance de l...,http://www.emb.cl/gerencia/noticia.mvc?nid=202...,emb.cl,unknown journalist,Chile,es,22985,0.0,2,Amazon,AI,Spanish South America,COUNTRIES,26,September,2023
4,db58ad4b2fd720d06fec378932051c3d0918efa4b2e91d...,2023-08-23 14:41:43,Revista Channel News - Mercado de servicios en...,http://www.emb.cl/channelnews/noticia.mvc?nid=...,emb.cl,unknown journalist,Chile,es,22985,0.0,2,Amazon,AI,Spanish South America,COUNTRIES,23,August,2023


In [104]:
# Change the column name "Outlet Name" of latam_q1 dataframe to "Outlet"
countries_q1.rename(columns={'Outlet Name':'Outlet'}, inplace=True)
print(countries_q1.shape)
countries_q1.head()

(11616, 18)


,"﻿""Article ID""",Published date,Headline,Url,Outlet,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region,Day,Month,Year
0,c8c49005158027f58df3198819e503989430d7ee16021f...,2023-07-27 07:23:00,AWS lanza siete nuevos cursos para aprender IA...,https://portalinnova.cl/aws-lanza-siete-nuevos...,portalinnova.cl,unknown journalist,Chile,es,4672,0.0,2,Amazon,AI,Spanish South America,COUNTRIES,27,July,2023
1,bac1b1365d3347b0c72135e102e11a7ead5b378fc3f26a...,2023-09-20 19:15:53,"Alexa ""casi humana"": Amazon presenta nueva ver...",https://www.df.cl/internacional/multinacionale...,df.cl,unknown journalist,Chile,es,45232,16.0,2,Amazon,AI,Spanish South America,COUNTRIES,20,September,2023
2,9fadbc44ddb0af58e501328f0255e25a04a89ab5d778dc...,2023-09-06 08:16:00,Grupo BMW selecciona a AWS para impulsar la pl...,https://portalinnova.cl/grupo-bmw-selecciona-a...,portalinnova.cl,Journalist Unknown,Chile,es,4672,0.0,2,Amazon,AI,Spanish South America,COUNTRIES,06,September,2023
3,6d4a013c3b84331acf1669b171ca9cf5ab7c06f384d7d9...,2023-09-26 17:53:09,Amazon y Anthropic colaboran en el avance de l...,http://www.emb.cl/gerencia/noticia.mvc?nid=202...,emb.cl,unknown journalist,Chile,es,22985,0.0,2,Amazon,AI,Spanish South America,COUNTRIES,26,September,2023
4,db58ad4b2fd720d06fec378932051c3d0918efa4b2e91d...,2023-08-23 14:41:43,Revista Channel News - Mercado de servicios en...,http://www.emb.cl/channelnews/noticia.mvc?nid=...,emb.cl,unknown journalist,Chile,es,22985,0.0,2,Amazon,AI,Spanish South America,COUNTRIES,23,August,2023


In [105]:
countries_q1_processed = countries_q1.merge(media_list_fy24, on='Outlet', how='left')
print(countries_q1_processed.shape)
countries_q1_processed.head()

(11616, 26)


,"﻿""Article ID""",Published date,Headline,Url,Outlet,Author Name,Country,Language,Impressions,Shares,...,Month,Year,Media Type,Region_y,Women,Men,18-24,25-34,35-44,45+
0,c8c49005158027f58df3198819e503989430d7ee16021f...,2023-07-27 07:23:00,AWS lanza siete nuevos cursos para aprender IA...,https://portalinnova.cl/aws-lanza-siete-nuevos...,portalinnova.cl,unknown journalist,Chile,es,4672,0.0,...,July,2023,Tech,Americas,0.433839,0.566161,0.184644,0.335787,0.231787,0.247783
1,bac1b1365d3347b0c72135e102e11a7ead5b378fc3f26a...,2023-09-20 19:15:53,"Alexa ""casi humana"": Amazon presenta nueva ver...",https://www.df.cl/internacional/multinacionale...,df.cl,unknown journalist,Chile,es,45232,16.0,...,September,2023,Business,Americas,0.429460,0.570540,0.144834,0.311942,0.238096,0.305128
2,9fadbc44ddb0af58e501328f0255e25a04a89ab5d778dc...,2023-09-06 08:16:00,Grupo BMW selecciona a AWS para impulsar la pl...,https://portalinnova.cl/grupo-bmw-selecciona-a...,portalinnova.cl,Journalist Unknown,Chile,es,4672,0.0,...,September,2023,Tech,Americas,0.433839,0.566161,0.184644,0.335787,0.231787,0.247783
3,6d4a013c3b84331acf1669b171ca9cf5ab7c06f384d7d9...,2023-09-26 17:53:09,Amazon y Anthropic colaboran en el avance de l...,http://www.emb.cl/gerencia/noticia.mvc?nid=202...,emb.cl,unknown journalist,Chile,es,22985,0.0,...,September,2023,Business,Americas,0.459346,0.540654,0.208164,0.330350,0.218612,0.242874
4,db58ad4b2fd720d06fec378932051c3d0918efa4b2e91d...,2023-08-23 14:41:43,Revista Channel News - Mercado de servicios en...,http://www.emb.cl/channelnews/noticia.mvc?nid=...,emb.cl,unknown journalist,Chile,es,22985,0.0,...,August,2023,Business,Americas,0.459346,0.540654,0.208164,0.330350,0.218612,0.242874


## Content Extraction - LATAM Q1 PROCESSED

In [ ]:
# Use concurrent.futures to process URLs concurrently
start = time.time()
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(process_url, latam_q1_processed['Url'])

latam_q1_processed['Content'] = list(results)
print(latam_q1_processed.shape)
latam_q1_processed.head()
end = time.time()
elapsed_time = (end - start) / 60
print("Elapsed time: {:.2f} minutes".format(elapsed_time))

print(latam_q1_processed.shape)
latam_q1_processed.head()

Error: Invalid header value in article download.
URL Content from https://olhardigital.com.br/2023/08/17/pro/numero-de-campanhas-politicas-manipuladas-usando-ia-cresceu-segundo-empresa-do-google/ is correct
Error: Invalid header value in article download.
URL Content from https://www.iproup.com/innovacion/42300-ia-generativa-asi-es-la-nueva-herramienta-que-lanzara-google is correct
URL Content from https://tecnoblog.net/noticias/2023/08/21/trafego-no-bing-esta-caindo-e-pode-ter-relacao-com-a-queda-no-hype-da-sua-ia/ is correct
URL Content from https://www.eltiempo.com/tecnosfera/novedades-tecnologia/google-fotos-lanza-pestana-de-recuerdos-y-es-organizadas-con-ia-797350 is correct
URL Content from https://www.semana.com/tecnologia/articulo/google-toma-la-delantera-ante-apple-al-lanzar-una-funcion-que-ni-su-dispositivo-mas-popular-tiene/202321/ is correct
URL Content from https://www.infobae.com/tecno/2023/07/04/google-une-el-legado-de-leonardo-da-vinci-con-la-inteligencia-artificial-y-e

In [ ]:
latam_q1_processed['Content_Length'] = latam_q1_processed['Content'].str.len()
latam_q1_processed.head()

latam_q1_processed['Content_Length_Check'] = np.where(latam_q1_processed['Content_Length'] < 400, 'CHECK', 'OK')
print(latam_q1_processed.shape)
latam_q1_processed.head()

## Content Extraction - COUNTRIES Q1 PROCESSED

In [ ]:
# Use concurrent.futures to process URLs concurrently
start = time.time()
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(process_url, countries_q1_processed['Url'])

countries_q1_processed['Content'] = list(results)
print(countries_q1_processed.shape)
countries_q1_processed.head()
end = time.time()
elapsed_time = (end - start) / 60
print("Elapsed time: {:.2f} minutes".format(elapsed_time))

print(countries_q1_processed.shape)
countries_q1_processed.head()

URL Content from https://www.df.cl/internacional/multinacionales/amazon-se-suma-a-la-carrera-de-los-chatbots-y-lanza-alexa-con-ia-generativa is correct
URL Content from https://www.emol.com/noticias/Tecnologia/2023/07/13/1100849/google-lanza-bard-inteligencia-artificial.html is correct
URL Content from https://portalinnova.cl/grupo-bmw-selecciona-a-aws-para-impulsar-la-plataforma-de-conduccion-automatizada-de-proxima-generacion/ is correct
URL Content from https://www.df.cl/opinion/columnistas/inteligencia-artificial-mas-alla-de-chat-gpt is correct
URL Content from https://www.itseller.cl/2023/09/27/las-claves-para-crecer-en-el-segmento-pymes-con-microsoft-y-licencias-online/ is correct
URL Content from https://www.df.cl/empresas/telecom-tecnologia/la-inteligencia-artificial-puede-ayudar-a-microsoft-a-sumarse-a-apple-en is correct
URL Content from https://portalinnova.cl/aws-lanza-siete-nuevos-cursos-para-aprender-ia-generativa/ is correct
URL Content from {} is correct from diffbot
Er

,"﻿""Article ID""",Published date,Headline,Url,Outlet,Author Name,Country,Language,Impressions,Shares,...,Year,Media Type,Region_y,Women,Men,18-24,25-34,35-44,45+,Content
0,c8c49005158027f58df3198819e503989430d7ee16021f...,2023-07-27 07:23:00,AWS lanza siete nuevos cursos para aprender IA...,https://portalinnova.cl/aws-lanza-siete-nuevos...,portalinnova.cl,unknown journalist,Chile,es,4672,0.0,...,2023,Tech,Americas,0.433839,0.566161,0.184644,0.335787,0.231787,0.247783,Las capacitaciones en habilidades de AWS sobre...
1,bac1b1365d3347b0c72135e102e11a7ead5b378fc3f26a...,2023-09-20 19:15:53,"Alexa ""casi humana"": Amazon presenta nueva ver...",https://www.df.cl/internacional/multinacionale...,df.cl,unknown journalist,Chile,es,45232,16.0,...,2023,Business,Americas,0.429460,0.570540,0.144834,0.311942,0.238096,0.305128,Javascript está deshabilitado en su navegador ...
2,9fadbc44ddb0af58e501328f0255e25a04a89ab5d778dc...,2023-09-06 08:16:00,Grupo BMW selecciona a AWS para impulsar la pl...,https://portalinnova.cl/grupo-bmw-selecciona-a...,portalinnova.cl,Journalist Unknown,Chile,es,4672,0.0,...,2023,Tech,Americas,0.433839,0.566161,0.184644,0.335787,0.231787,0.247783,El fabricante líder de automóviles de lujo est...
3,6d4a013c3b84331acf1669b171ca9cf5ab7c06f384d7d9...,2023-09-26 17:53:09,Amazon y Anthropic colaboran en el avance de l...,http://www.emb.cl/gerencia/noticia.mvc?nid=202...,emb.cl,unknown journalist,Chile,es,22985,0.0,...,2023,Business,Americas,0.459346,0.540654,0.208164,0.330350,0.218612,0.242874,Something went wrong with url
4,db58ad4b2fd720d06fec378932051c3d0918efa4b2e91d...,2023-08-23 14:41:43,Revista Channel News - Mercado de servicios en...,http://www.emb.cl/channelnews/noticia.mvc?nid=...,emb.cl,unknown journalist,Chile,es,22985,0.0,...,2023,Business,Americas,0.459346,0.540654,0.208164,0.330350,0.218612,0.242874,Something went wrong with url


In [111]:
countries_q1_processed['Content_Length'] = countries_q1_processed['Content'].str.len()
countries_q1_processed.head()

countries_q1_processed['Content_Length_Check'] = np.where(countries_q1_processed['Content_Length'] < 400, 'CHECK', 'OK')
print(countries_q1_processed.shape)
countries_q1_processed.head()

(11616, 29)


,"﻿""Article ID""",Published date,Headline,Url,Outlet,Author Name,Country,Language,Impressions,Shares,...,Region_y,Women,Men,18-24,25-34,35-44,45+,Content,Content_Length,Content_Length_Check
0,c8c49005158027f58df3198819e503989430d7ee16021f...,2023-07-27 07:23:00,AWS lanza siete nuevos cursos para aprender IA...,https://portalinnova.cl/aws-lanza-siete-nuevos...,portalinnova.cl,unknown journalist,Chile,es,4672,0.0,...,Americas,0.433839,0.566161,0.184644,0.335787,0.231787,0.247783,Las capacitaciones en habilidades de AWS sobre...,4763.0,OK
1,bac1b1365d3347b0c72135e102e11a7ead5b378fc3f26a...,2023-09-20 19:15:53,"Alexa ""casi humana"": Amazon presenta nueva ver...",https://www.df.cl/internacional/multinacionale...,df.cl,unknown journalist,Chile,es,45232,16.0,...,Americas,0.429460,0.570540,0.144834,0.311942,0.238096,0.305128,Javascript está deshabilitado en su navegador ...,2624.0,OK
2,9fadbc44ddb0af58e501328f0255e25a04a89ab5d778dc...,2023-09-06 08:16:00,Grupo BMW selecciona a AWS para impulsar la pl...,https://portalinnova.cl/grupo-bmw-selecciona-a...,portalinnova.cl,Journalist Unknown,Chile,es,4672,0.0,...,Americas,0.433839,0.566161,0.184644,0.335787,0.231787,0.247783,El fabricante líder de automóviles de lujo est...,9807.0,OK
3,6d4a013c3b84331acf1669b171ca9cf5ab7c06f384d7d9...,2023-09-26 17:53:09,Amazon y Anthropic colaboran en el avance de l...,http://www.emb.cl/gerencia/noticia.mvc?nid=202...,emb.cl,unknown journalist,Chile,es,22985,0.0,...,Americas,0.459346,0.540654,0.208164,0.330350,0.218612,0.242874,Something went wrong with url,29.0,CHECK
4,db58ad4b2fd720d06fec378932051c3d0918efa4b2e91d...,2023-08-23 14:41:43,Revista Channel News - Mercado de servicios en...,http://www.emb.cl/channelnews/noticia.mvc?nid=...,emb.cl,unknown journalist,Chile,es,22985,0.0,...,Americas,0.459346,0.540654,0.208164,0.330350,0.218612,0.242874,Something went wrong with url,29.0,CHECK


## Save Files - Repository

In [137]:
print(latam_q1_processed.shape)
latam_q1_processed.head()

(10280, 29)


,"﻿""Article ID""",Published date,Headline,Url,Outlet,Author Name,Country,Language,Impressions,Shares,...,Region_y,Women,Men,18-24,25-34,35-44,45+,Content,Content_Length,Content_Length_Check
0,c17a11aa3fc98c1024cae20f080c6ee8fbd10464d844af...,2023-08-17 13:29:00,IA es un “maremoto” tan grande como internet: ...,https://www.elespectador.com/tecnologia/ia-es-...,elespectador.com,unknown journalist,Colombia,es,329107,3.0,...,Americas,0.458474,0.541526,0.255116,0.350201,0.170343,0.224340,En el centro del intento más reciente por tran...,3116.0,OK
1,7bd7c72adaebfe21a2f49154e2bca8a5b8e04610cf3e28...,2023-08-18 09:00:00,IA Generativa: Google lanza una nueva herramie...,https://www.iproup.com/innovacion/42300-ia-gen...,iproup.com,unknown journalist,Argentina,es,7552,11.0,...,Americas,0.322019,0.677981,0.161540,0.328019,0.223239,0.287202,IA Generativa: Google lanza una nueva herramie...,3992.0,OK
2,2f09d981e4dfd063a3bd152502768bfae605d08a5c6deb...,2023-08-17 18:35:00,Número de campanhas políticas manipuladas usan...,https://olhardigital.com.br/2023/08/17/pro/num...,olhardigital.com.br,Vitoria Lopes Gomez,Brazil,pt,209607,2.0,...,Americas,0.361911,0.638089,0.255087,0.348724,0.178670,0.217519,A preocupação com o impacto da inteligência ar...,3242.0,OK
3,b56b1ddf9e15b478f66cbb0b395f3eb2888cedfcf95b44...,2023-08-11 09:25:00,IA generativa: Magalu e Dasa apresentam projet...,https://itforum.com.br/noticias/ia-generativa-...,itforum.com.br,Marcelo Gimenes Vieira,Brazil,pt,21534,11.0,...,Americas,0.360594,0.639406,0.244546,0.391039,0.200255,0.164160,"Quando Andre Fatala, vice-presidente de tecnol...",4873.0,OK
4,ed2b4ac6329dd3d77ea6400107cd1f4f0779bf5a568692...,2023-08-22 17:09:43,Google se la juega: Bard reveló cuál es la mej...,https://www.iproup.com/economia-digital/42067-...,iproup.com,unknown journalist,Argentina,es,7552,32.0,...,Americas,0.322019,0.677981,0.161540,0.328019,0.223239,0.287202,None,NaN,OK


In [138]:
print(countries_q1_processed.shape)
countries_q1_processed.head()

(11616, 29)


,"﻿""Article ID""",Published date,Headline,Url,Outlet,Author Name,Country,Language,Impressions,Shares,...,Region_y,Women,Men,18-24,25-34,35-44,45+,Content,Content_Length,Content_Length_Check
0,c8c49005158027f58df3198819e503989430d7ee16021f...,2023-07-27 07:23:00,AWS lanza siete nuevos cursos para aprender IA...,https://portalinnova.cl/aws-lanza-siete-nuevos...,portalinnova.cl,unknown journalist,Chile,es,4672,0.0,...,Americas,0.433839,0.566161,0.184644,0.335787,0.231787,0.247783,Las capacitaciones en habilidades de AWS sobre...,4763.0,OK
1,bac1b1365d3347b0c72135e102e11a7ead5b378fc3f26a...,2023-09-20 19:15:53,"Alexa ""casi humana"": Amazon presenta nueva ver...",https://www.df.cl/internacional/multinacionale...,df.cl,unknown journalist,Chile,es,45232,16.0,...,Americas,0.429460,0.570540,0.144834,0.311942,0.238096,0.305128,Javascript está deshabilitado en su navegador ...,2624.0,OK
2,9fadbc44ddb0af58e501328f0255e25a04a89ab5d778dc...,2023-09-06 08:16:00,Grupo BMW selecciona a AWS para impulsar la pl...,https://portalinnova.cl/grupo-bmw-selecciona-a...,portalinnova.cl,Journalist Unknown,Chile,es,4672,0.0,...,Americas,0.433839,0.566161,0.184644,0.335787,0.231787,0.247783,El fabricante líder de automóviles de lujo est...,9807.0,OK
3,6d4a013c3b84331acf1669b171ca9cf5ab7c06f384d7d9...,2023-09-26 17:53:09,Amazon y Anthropic colaboran en el avance de l...,http://www.emb.cl/gerencia/noticia.mvc?nid=202...,emb.cl,unknown journalist,Chile,es,22985,0.0,...,Americas,0.459346,0.540654,0.208164,0.330350,0.218612,0.242874,Something went wrong with url,29.0,CHECK
4,db58ad4b2fd720d06fec378932051c3d0918efa4b2e91d...,2023-08-23 14:41:43,Revista Channel News - Mercado de servicios en...,http://www.emb.cl/channelnews/noticia.mvc?nid=...,emb.cl,unknown journalist,Chile,es,22985,0.0,...,Americas,0.459346,0.540654,0.208164,0.330350,0.218612,0.242874,Something went wrong with url,29.0,CHECK


In [139]:
latam_q1_processed.to_excel('sagemaker_results/latam_categories_q1_processed.xlsx', index=False, engine='xlsxwriter')

In [ ]:
countries_q1_processed.to_excel('sagemaker_results/countries_categories_q1_processed.xlsx', index=False, engine='xlsxwriter')

## Save Processed DataFrames in AWS S3

In [123]:
# Let's define the S3 bucket and prefix that we want to use in this session
bucket = 'ms-cims-fy24' # bucket name
prefix = 'sagemaker_results' # prefix is the subfolder within the folder. To specify this region: US East (Ohio) us-east-2

# Specify the region
region = 'us-east-2'

In [124]:
import sagemaker
role = sagemaker.get_execution_role()
print(role)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
arn:aws:iam::756970149474:role/service-role/AmazonSageMaker-ExecutionRole-20231018T143251


In [125]:
import io
import boto3
import pandas as pd


# Create an S3 client
s3 = boto3.client('s3')

# Create a buffer
buf = io.BytesIO()

# Write the DataFrame to the buffer
latam_q1_processed.to_csv(buf, index=False)

# Seek to the beginning of the buffer
buf.seek(0)

# Upload the DataFrame to S3
s3.put_object(Bucket= bucket, Key='latam_q1_processed.csv', Body=buf)


{'ResponseMetadata': {'RequestId': 'G4HCKV3Z4HB1TCCG',
  'HostId': '3aoy+0sQjiTSHASfSHExWLez4SlaJZZ1WH8nRVWw4QJwr3UKGexYoW3kycSHnidcN+tG99XPWEI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '3aoy+0sQjiTSHASfSHExWLez4SlaJZZ1WH8nRVWw4QJwr3UKGexYoW3kycSHnidcN+tG99XPWEI=',
   'x-amz-request-id': 'G4HCKV3Z4HB1TCCG',
   'date': 'Tue, 24 Oct 2023 21:21:19 GMT',
   'x-amz-version-id': 'RLEbyy6o7425k3f.zVPBWhxf4bcXP7HU',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"6369fbefdcebe75b299d4e400bd4cedc"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"6369fbefdcebe75b299d4e400bd4cedc"',
 'ServerSideEncryption': 'AES256',
 'VersionId': 'RLEbyy6o7425k3f.zVPBWhxf4bcXP7HU'}

In [126]:
import os

# Code to upload RecordIO data to S3
key_latam = 'latam-q1-processed'

# The following code uploads the data in record-io format to S3 bucket to be accessed later for training
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'latam', key_latam)).upload_fileobj(buf)

# Let's print out the training data location in S3
latam_q1_processed = 's3://{}/{}/train/{}'.format(bucket, prefix, key_latam)
print('uploaded q1 processed latam categories data location: {}'.format(latam_q1_processed))

uploaded q1 processed latam categories data location: s3://ms-cims-fy24/sagemaker_results/train/latam-q1-processed
